In [1]:
import numpy as np
import faiss
import time
import torch
import pandas as pd
from os import path, makedirs
import matplotlib.pyplot as plt
import sys
sys.path.append('../')
from utils import load_embeddings

## CONFIG

In [3]:
model = 'mrl' # mrl, rr
arch = 'resnet50' # resnet18, resnet34, resnet50, resnet101, mobilenetv2
root = f'../../../inference_array/{arch}'
dataset = '1K' # 1K, 4K, V2, inat

In [4]:
index_type = 'ivf' # ivfpq, ivfsq, lsh, pq, ivf, opq

hnsw_max_neighbors = 32 # 8, 32

k = 2048 # shortlist length, default set to max supported by FAISS

nesting_list_dict = {
    'vgg19':  [1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048, 4096],
    'resnet18': [1, 2, 4, 8, 16, 32, 64, 128, 256, 512],
    'resnet34': [1, 2, 4, 8, 16, 32, 64, 128, 256, 512],
    'resnet50': [1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048],
    'resnet101': [1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048],
    'mobilenetv2': [10, 20, 40, 80, 160, 320, 640, 1280],
    'convnext_tiny': [12, 24, 48, 96, 192, 384, 768]
}

nesting_list = nesting_list_dict[arch]

ivf_configs = {
    'C1': {'nlist': 2048},
    'C2': {'nlist': 512},
    'C3': {'nlist': 128},
    'C4': {'nlist': 4096},
    'C5': {'nlist': 8192},
    'C6': {'nlist': 256},
    'C7': {'nlist': 1024}
}

config_id = 'C7'
    
# nbits = ivfpq_configs[config_id]['nbits'] # nbits used to represent centroid id; total possible is k* = 2**nbits

nlist = ivf_configs[config_id]['nlist']  # how many Voronoi cells (must be >= k*)

In [5]:
if index_type == 'exactl2':
    use_gpu = 1 # GPU inference for exact search
else:
    use_gpu = 0

if use_gpu and faiss.get_num_gpus() == 0:
    raise Exception("GPU search is enabled but no GPU was found.")

## IVF Experiments

In [7]:
Ds = [8, 16, 32, 64, 128, 256, 512, 1024, 2048]
num_cell_list = [256, 512, 1024, 2048, 4096, 8192] # IVF number of clusters
nprobes = [1, 2, 4]

for D in Ds:
    
    database, queryset, _, _, _, _ = load_embeddings(model, dataset, D, arch=arch)

    num_cell_list = [256, 512, 1024, 2048, 4096, 8192] # IVF num of clusters
    for num_cell in num_cell_list:
        if not path.isdir(root+'index_files/'+model+'/'):
            makedirs(root+'index_files/'+model+'/')

        print("Indexing database: ", database.shape)
        d = database.shape[1]
        index_file = root+'index_files/'+model+'/'+dataset+'_ivf'+'_nlist'+str(num_cell)+"_d"+str(d)
        
        # Load or build index
        if path.exists(index_file+'.index'):
            print("Loading index file: ” + index_file)
            cpu_index = faiss.read_index(index_file+'.index')
        else:
            if index_type == 'ivf':
                print("Generating index file: ” + index_file)
                # Generate IVF Index File
                quantizer = faiss.IndexFlatL2(d) # L2 quantizer to assign vectors to Voronoi cells
                cpu_index = faiss.IndexIVFFlat(quantizer, d, num_cell)
                print(f”Training IVF Index: d={d}, nlist={num_cell}")
                ivf_start = time.time()
                cpu_index.train(database)
                ivf_end = time.time()
                print("IVF Index train time= %0.3f sec” % (ivf_end - ivf_start))
            elif index_type == 'exactl2':
                print("Building Exact L2 Index”)
                cpu_index = faiss.IndexFlatL2(d)
            cpu_index.add(database)                  # add vectors to the index
            faiss.write_index(cpu_index, index_file+'.index')
        if use_gpu:
            index = faiss.index_cpu_to_all_gpus(cpu_index)
        else:
            index = cpu_index
        for nprobe in nprobes: # nprobe for IVF
            print(f”NProbe: {nprobe}")
            print("Searching queries: ", queryset.shape)
            start = time.time()
            index.nprobe = nprobe
            Dist, Ind = index.search(queryset, k) # k->shorlist length
            end = time.time() - start
            print("%d-NN search time= %f sec\n” % (k, end))
            if not path.isdir(root+"neighbors/"+model):
                makedirs(root+"neighbors/"+model)
            nn_dir = root+"neighbors/"+model+index_type+"_nlist”+str(num_cell)+f”_nprobe{nprobe}_"+str(k)+"shortlist_"+dataset+f”_d{d}.csv”
            pd.DataFrame(Ind).to_csv(nn_dir, header=None, index=None)
            print("Writing NN csv to %s\n” % (nn_dir))
        del index
        print("*************************************************")

*************************************************
FF-8 IVF 

*************************************************
Loaded DB: 1K_train_mrl0_e0_ff8-X.npy
DB size: 39.10 MB
DB: (1281167, 8), Q: (50000, 8)
Indexing database:  (1281167, 8)
Generating index file: ../../../inference_array/resnet50/index_files/ff-ivf/8/1K_ivf_nlist256_d8
Training IVF Index: d=8, nlist=256
IVF Index train time= 0.141 sec
NProbe: 1
Searching queries:  (50000, 8)
GPU 2048-NN search time= 2.962470 sec

Writing NN csv to ../../../inference_array/resnet50/neighbors/ff-ivf/8/ivf_nlist256_nprobe1_2048shortlist_1K_d8.csv

*************************************************
Indexing database:  (1281167, 8)
Generating index file: ../../../inference_array/resnet50/index_files/ff-ivf/8/1K_ivf_nlist512_d8
Training IVF Index: d=8, nlist=512
IVF Index train time= 0.272 sec
NProbe: 1
Searching queries:  (50000, 8)
GPU 2048-NN search time= 2.063317 sec

Writing NN csv to ../../../inference_array/resnet50/neighbors/ff-ivf/8/ivf_nlist

IVF Index train time= 397.688 sec
NProbe: 1
Searching queries:  (50000, 512)
GPU 2048-NN search time= 2.997688 sec

Writing NN csv to ../../../inference_array/resnet50/neighbors/ff-ivf/512/ivf_nlist8192_nprobe1_2048shortlist_1K_d512.csv

*************************************************
